In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-02-16 12:42:38--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-02-16 12:42:38--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-02-16 12:42:38--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [2]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
import itertools
import nltk
import torch
import torch.nn as nn
from torchvision import datasets

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
train_path = "/content/drive/MyDrive/anlp_final/snli_1.0_dev.csv"
df = pd.read_csv(train_path)
df.head()

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5
0,neutral,( ( Two women ) ( ( are ( embracing ( while ( ...,( ( The sisters ) ( ( are ( ( hugging goodbye ...,(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP ar...,(ROOT (S (NP (DT The) (NNS sisters)) (VP (VBP ...,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...,4705552913.jpg#2,4705552913.jpg#2r1n,neutral,entailment,neutral,neutral,neutral
1,entailment,( ( Two women ) ( ( are ( embracing ( while ( ...,( ( Two woman ) ( ( are ( holding packages ) )...,(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP ar...,(ROOT (S (NP (CD Two) (NN woman)) (VP (VBP are...,Two women are embracing while holding to go pa...,Two woman are holding packages.,4705552913.jpg#2,4705552913.jpg#2r1e,entailment,entailment,entailment,entailment,entailment
2,contradiction,( ( Two women ) ( ( are ( embracing ( while ( ...,( ( The men ) ( ( are ( fighting ( outside ( a...,(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP ar...,(ROOT (S (NP (DT The) (NNS men)) (VP (VBP are)...,Two women are embracing while holding to go pa...,The men are fighting outside a deli.,4705552913.jpg#2,4705552913.jpg#2r1c,contradiction,contradiction,contradiction,contradiction,contradiction
3,entailment,( ( ( Two ( young children ) ) ( in ( ( ( ( ( ...,( ( ( Two kids ) ( in ( numbered jerseys ) ) )...,(ROOT (S (NP (NP (CD Two) (JJ young) (NNS chil...,(ROOT (S (NP (NP (CD Two) (NNS kids)) (PP (IN ...,"Two young children in blue jerseys, one with t...",Two kids in numbered jerseys wash their hands.,2407214681.jpg#0,2407214681.jpg#0r1e,entailment,entailment,entailment,entailment,entailment
4,neutral,( ( ( Two ( young children ) ) ( in ( ( ( ( ( ...,( ( ( Two kids ) ( at ( a ballgame ) ) ) ( ( w...,(ROOT (S (NP (NP (CD Two) (JJ young) (NNS chil...,(ROOT (S (NP (NP (CD Two) (NNS kids)) (PP (IN ...,"Two young children in blue jerseys, one with t...",Two kids at a ballgame wash their hands.,2407214681.jpg#0,2407214681.jpg#0r1n,neutral,neutral,neutral,entailment,entailment


In [6]:
df = df[['sentence1', 'sentence2', 'gold_label']]
df.head()

,sentence1,sentence2,gold_label
0,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...,neutral
1,Two women are embracing while holding to go pa...,Two woman are holding packages.,entailment
2,Two women are embracing while holding to go pa...,The men are fighting outside a deli.,contradiction
3,"Two young children in blue jerseys, one with t...",Two kids in numbered jerseys wash their hands.,entailment
4,"Two young children in blue jerseys, one with t...",Two kids at a ballgame wash their hands.,neutral


In [7]:
df['gold_label'].value_counts()

entailment       3329
contradiction    3278
neutral          3235
-                 158
Name: gold_label, dtype: int64

In [8]:
def encode_label(df):
  df = df[['sentence1', 'sentence2', 'gold_label']]
  lb = LabelEncoder()
  df = df[(df['gold_label'] == 'neutral') | (df['gold_label'] == 'contradiction') | (df['gold_label'] == 'entailment')]
  df['labels'] = lb.fit_transform(df['gold_label'])
  classes_names = lb.classes_
  df.drop('gold_label', axis = 1,  inplace= True)
  return df, classes_names

In [9]:
df, classes_names = encode_label(df)

In [10]:
df.head()

,sentence1,sentence2,labels
0,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...,2
1,Two women are embracing while holding to go pa...,Two woman are holding packages.,1
2,Two women are embracing while holding to go pa...,The men are fighting outside a deli.,0
3,"Two young children in blue jerseys, one with t...",Two kids in numbered jerseys wash their hands.,1
4,"Two young children in blue jerseys, one with t...",Two kids at a ballgame wash their hands.,2


In [11]:
classes_names

array(['contradiction', 'entailment', 'neutral'], dtype=object)

##Embedding Things

In [12]:
def Embedding_dict(path):
  emb_dct = {}
  with open(path) as f:
    for line in f:
      word, coefs = line.split(maxsplit=1)
      coefs = np.fromstring(coefs, "f", sep=" ")
      emb_dct[word] = coefs
  print(f"Number of words found are {len(emb_dct)}")
  return emb_dct

In [13]:
def create_dataset(df, dct, func = 'sum', emb_dim = 300):
  embedding_matrix = np.zeros((df.shape[0], 2*emb_dim))
  hit = 0
  miss = 0
  for i, (s1, s2) in enumerate(zip(df['sentence1'], df['sentence2'])):
    vec1 = []
    vec2 = []
    s1 = s1.lower()
    s2 = s2.lower()
    s1 = word_tokenize(s1)
    s2 = word_tokenize(s2)
    for w1, w2 in list(itertools.zip_longest(s1, s2, fillvalue='foo')):
      if w1 != "foo":
        check1 = dct.get(w1)
        if check1 is not None:
          vec1.append(check1)
          hit+=1
        else:
          miss+=1
        
      if w2 != "foo":
        check1 = dct.get(w2)
        if check1 is not None:
          vec2.append(check1)
          hit+=1
        else:
          miss+=1
    ##
    vec1 = np.asarray(vec1)
    vec2 = np.asarray(vec2)
    if func == "sum":
      vec1 = np.sum(vec1, axis = 0)
      vec2 = np.sum(vec2, axis = 0)
    if func == "mean":
      vec1 = np.mean(vec1, axis = 0)
      vec2 = np.mean(vec2, axis = 0)
    #print("vec1 ", vec1.shape, "vec2 shape ", vec2.shape, "i is ", i)
    embedding_matrix[i, :] = np.concatenate((vec1, vec2))
  ##
  print(f"The number of hits are {hit} and the number of miss are {miss}")
  labels = df['labels'].values
  return embedding_matrix, labels    

In [ ]:
class simple_model(nn.Module):
  def __init__(self):
    super(simple_model, self).__init__()
    # self.embedding_matrix = embedding_matrix
    # self.labels = labels
    self.fc1 = nn.Linear(600, 256)
    self.fc2 = nn.Linear(256, 64)
    self.relu = nn.ReLU()
    self.fc3 = nn.Linear(64, 3)

  def forward(self, embedding_matrix):
    x = self.fc1(embedding_matrix)
    x = self.relu(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.fc3(x)
    return x

In [ ]:
def train(model, embedding_matrix, labels, batch_size = 32, lr = 0.007, epochs = 4):
  model.train()
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr) 
  #model.zero_grad()
  for epoch in range(epochs):
    l = 0
    for i in range(embedding_matrix.shape[0]//batch_size):
      optimizer.zero_grad()
      inp = embedding_matrix[i*batch_size: (i*batch_size + batch_size), :]
      lab = torch.from_numpy(labels[i*batch_size: (i*batch_size + batch_size)])
      out = model(torch.from_numpy(inp).float())
      loss = criterion(out, lab)
      l += loss
      loss.backward()
      optimizer.step()

      
    if epoch%10 == 0:
      print("loss is ", loss)
      print(f"for epoch {epoch} loss is {l}")

In [ ]:
model = simple_model()

In [ ]:
train(model, embedding_matrix, labels, lr = 0.005, epochs = 500)

In [ ]:
ypredt = model(torch.from_numpy(embedding_matrix).float())

In [ ]:
ypredt = torch.argmax(ypredt, dim = 1).numpy()

In [ ]:
labels

array([2, 1, 0, ..., 1, 0, 2])

In [ ]:
ypredt == labels

array([ True,  True,  True, ...,  True, False,  True])

In [ ]:
np.sum(ypredt == labels)/len(ypredt)

0.8984962406015038

In [ ]:
# dct = Embedding_dict("/content/glove.6B.300d.txt")

Number of words found are 400000


In [ ]:
# embedding_matrix, labels = create_dataset(df, dct)

The number of hits are 211049 and the number of miss are 20614


In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/anlp_final/snli_1.0_test.csv")
df_test, class_test = encode_label(df_test)

In [ ]:
df_test.head()

,sentence1,sentence2,labels
0,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,2
1,This church choir sings to the masses as they ...,The church is filled with song.,1
2,This church choir sings to the masses as they ...,A choir singing at a baseball game.,0
3,"A woman with a green headscarf, blue shirt and...",The woman is young.,2
4,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,1


In [ ]:
xte, yte = create_dataset(df_test, dct)

The number of hits are 229983 and the number of miss are 598


In [ ]:
xte[:5,:]

array([[-5.13825274, -1.14646089, -2.48438668, ..., -1.49156785,
        -0.68954498,  0.13984601],
       [-5.13825274, -1.14646089, -2.48438668, ..., -0.68161893,
        -1.6593082 , -0.36632299],
       [-5.13825274, -1.14646089, -2.48438668, ..., -0.93746603,
        -1.24169397,  0.06316002],
       [-2.2857213 , -0.36229506,  0.77572596, ...,  0.1158911 ,
        -1.12795997,  0.78471696],
       [-2.2857213 , -0.36229506,  0.77572596, ..., -0.4090839 ,
        -0.66159904,  1.14149594]])

In [ ]:
xte = torch.from_numpy(xte).float()
yte =  torch.from_numpy(yte)

In [ ]:
ypred = model(xte)

In [ ]:
ypred[-10:,:]

tensor([[ 4.3126e+01,  7.0347e+01, -1.0076e+01],
        [-1.0572e+01,  2.7790e+01,  1.8030e+01],
        [-1.9625e+00,  5.8161e+00,  2.8308e+00],
        [ 9.4442e-02,  5.6617e-01, -8.0060e-01],
        [ 1.4751e+00,  1.1778e-01,  1.1701e+00],
        [ 3.4894e+01,  3.0397e+01, -6.7366e+00],
        [ 4.0253e+00,  5.8051e+00, -2.0695e+00],
        [ 2.6350e+02,  4.5084e+01,  8.0810e+01],
        [ 6.1699e+00,  1.8057e+01,  5.8050e+00],
        [-6.3174e+00, -2.1244e+01,  4.0846e+01]], grad_fn=<SliceBackward>)

In [ ]:
ypred = torch.argmax(ypred, dim = 1)

In [ ]:
ypred = ypred.numpy()

In [ ]:
yte = yte.numpy()

In [ ]:
np.sum(ypred == yte)/len(yte)

0.5315553745928339

## LSTM

In [15]:
a = np.array([[1,2,3], [4,5,6]])
b = np.array([[7,8,9], [10,11,12]])
np.concatenate((a,b))

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [21]:
def create_dataset_for_sequence(df, dct, emb_dim = 300, seq_len = 20):
  sequence_matrix = np.zeros((df.shape[0], seq_len*2 ,emb_dim))
  hit = 0
  miss = 0
  for i, (s1, s2) in enumerate(zip(df['sentence1'], df['sentence2'])):
    vec1 = np.zeros((seq_len, emb_dim))
    vec2 = np.zeros((seq_len, emb_dim))
    s1 = s1.lower()
    s2 = s2.lower()
    s1 = word_tokenize(s1)
    s2 = word_tokenize(s2)
    for j, (w1, w2) in enumerate(list(itertools.zip_longest(s1, s2, fillvalue='foo'))):
      if j>= seq_len:
        break
      if w1 != "foo":
        check1 = dct.get(w1)
        if check1 is not None:
          vec1[j, :] = check1
          hit+=1
        else:
          miss+=1
        
      if w2 != "foo":
        check1 = dct.get(w2)
        if check1 is not None:
          vec2[j, :] = check1
          hit+=1
        else:
          miss+=1
    ##
    # vec1 = np.asarray(vec1)
    # vec2 = np.asarray(vec2)
    # if func == "sum":
    #   vec1 = np.sum(vec1, axis = 0)
    #   vec2 = np.sum(vec2, axis = 0)
    # if func == "mean":
    #   vec1 = np.mean(vec1, axis = 0)
    #   vec2 = np.mean(vec2, axis = 0)
    #print("vec1 ", vec1.shape, "vec2 shape ", vec2.shape, "i is ", i)
    sequence_matrix[i, :, :] = np.concatenate((vec1, vec2))
  ##
  print(f"The number of hits are {hit} and the number of miss are {miss}")
  labels = df['labels'].values
  return sequence_matrix, labels    

In [45]:
class Mylstm(nn.Module):
  def __init__(self, batch_size, hidden_size, num_classes, num_layers, embedding_dim):
    super(Mylstm, self).__init__()
    self.batch_size = batch_size
    self.hidden_size = hidden_size
    self.num_classes = num_classes
    self.num_layers = num_layers
    self.embedding_dim = embedding_dim
    self.lstm = nn.LSTM(input_size= self.embedding_dim, hidden_size= self.hidden_size, num_layers = self.num_layers, batch_first = True)
    self.fc = nn.Linear(self.hidden_size, self.num_classes)
    

  # def forward(self, x):
  #   out, (self.h0, self.c0) = self.lstm(x, (self.h0, self.c0))
  #   out = self.fc(out[:,-1,:])
  #   return out


  def forward(self, x):
    out, (self.h0, self.c0) = self.lstm(x, (self.h0, self.c0))
    # out = out.contiguous().view(-1, self.hidden_size)
    tmp = out.clone()
    tmp = out[:, -1, :]
    #out = out[:,-1,:]
    #out = out.reshape(out.size(0)*out.size(1), out.size(2))
    out = self.fc(tmp)
    return out

  
  def init_hidden(self):
    self.h0 = torch.rand(1, self.batch_size, self.hidden_size)
    self.c0 = torch.rand(1, self.batch_size, self.hidden_size)


In [52]:
def train_lstm(model, seq, labels, epochs = 10, lr = 0.005, batch_size = 461, hidden_size = 2):
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr= lr)
  l = []
  for epoch in range(epochs):
    model.init_hidden()
    for i in range(batch_size, seq.shape[0], batch_size):
      model.zero_grad()
      xtrain = seq[i-batch_size: i,:,:]
      target = labels[i-batch_size: i]
      model.train()
      optimizer.zero_grad()
      out= model(xtrain)
      loss = criterion(out, target)
      print("")
      print("Loss here : ", loss.item())
      l.append(loss)
      loss.backward(retain_graph=True)
      optimizer.step()
      # hidden[0].detach()
      # hidden[1].detach()
    print("loss is ", l[:5])

In [20]:
dct = Embedding_dict("/content/glove.6B.300d.txt")

Number of words found are 400000


In [22]:
df_test = df.head(100)
df_test, label = create_dataset_for_sequence(df_test, dct)

The number of hits are 2358 and the number of miss are 6


In [28]:
df_test = torch.tensor(df_test).float()
label = torch.tensor(label).type(torch.LongTensor)

In [47]:
model = Mylstm(batch_size= 1, hidden_size= 128, num_classes= 3, num_layers= 1, embedding_dim= 300)

In [42]:
df_test.shape

torch.Size([100, 40, 300])

In [51]:
train_lstm(model, df_test, label, batch_size=1)

xtrain shape  torch.Size([1, 40, 300])

Loss here :  0.9972867965698242
xtrain shape  torch.Size([1, 40, 300])

Loss here :  1.191184163093567


RuntimeError: ignored